In [1]:
# Manipulação de dados
import duckdb
import pandas as pd

# Tratamento de dados
from scipy import stats
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Algoritmos
from xgboost import XGBRegressor, XGBClassifier

# Métricas de validação
from metrics import MAE,RMSE,R2_determinacao,PSC_A,PCC_A,PMC_A

# Salvar e importar modelos e métricas
import pickle
import json

# Configurações de bibliotecas
pd.set_option('display.max_columns',None)

# Importando o database de dados de modelagem
final_db = duckdb.connect(database='final_db')

# Defininindo os tipos de variáveis presentes no dataset
n_vizinhas_disponiveis = 10

list_key = ['id_estacao','dt_medicao']

list_features_geoespaciais = ['latitude','longitude','vl_declividade','vl_altitude','vl_distancia_oceano','vl_aspecto_relevo']

list_features_estacoes_temperatura = ['vl_temperatura_maxima','vl_temperatura_media','vl_temperatura_minima']
list_features_estacoes_umidade = ['vl_umidade_relativa_maxima','vl_umidade_relativa_media','vl_umidade_relativa_minima']
list_features_estacoes_vento = ['vl_velocidade_vento_2m_maxima','vl_velocidade_vento_2m_media','vl_velocidade_vento_10m_media']
list_features_estacoes = list_features_estacoes_temperatura + list_features_estacoes_umidade + list_features_estacoes_vento

list_features_chirps = ['vl_precipitacao_chirps']
list_features_cpc = ['vl_precipitacao_cpc','vl_temperatura_maxima_cpc','vl_temperatura_minima_cpc']
list_features_gpm_final_run = ['vl_precipitacao_gpm_final_run']
list_features_gpm_late_run = ['vl_precipitacao_gpm_late_run']
list_features_power = ['vl_precipitacao_power','vl_temperatura_maxima_2m_K_power','vl_temperatura_media_2m_K_power','vl_temperatura_minima_2m_K_power','vl_umidade_relativa_2m_power','vl_pressao_nivel_superficie_power','vl_irradiancia_allsky_power','vl_direcao_vento_10m_power','vl_direcao_vento_2m_power','vl_temperatura_orvalho_2m_K_power','vl_vento_10m_power','vl_vento_medio_2m_power','vl_vento_maximo_2m_power','vl_vento_maximo_10m_power']
list_features_produtos = list_features_chirps + list_features_cpc + list_features_gpm_final_run + list_features_gpm_late_run + list_features_power

list_vizinhas_aux = sum([[f'vl_correlacao_estacao_vizinha_{i}',f'pct_intersecao_precipitacao_vizinha_{i}',f'vl_distancia_km_vizinha_{i}',f'vl_prioridade_vizinha_{i}',f'vl_precipitacao_vizinha_{i}']for i in range(1, n_vizinhas_disponiveis+1)], [])
list_features_vizinhas = [f'vl_precipitacao_vizinha_{i}' for i in range(1,n_vizinhas_disponiveis+1)]

list_features_target = ['vl_precipitacao','vl_precipitacao_log']

list_split_column = ['percentil_temporal']

# MODELOS 1

In [10]:
print("""Os modelos 1 são os mais simples. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação, e não há tentativas 
de modificação da target\n
> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Sem modificações da target
> Testando diferentes combinações de variáveis explicativas
""")

Os modelos 1 são os mais simples. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação, e não há tentativas 
de modificação da target

> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Sem modificações da target
> Testando diferentes combinações de variáveis explicativas



In [3]:
def ImportBase_Modelo1(final_db=final_db,table_name='abt_base'):
    df = final_db.execute(f"""
    SELECT
        *
    FROM {table_name}
    """).fetch_df()

    return df

def SplitTreinoTeste_Modelo1(df_base,pct_split,coluna_percentil_temporal='percentil_temporal'):
  df_treino = df_base.loc[df_base[coluna_percentil_temporal]<=pct_split]
  df_teste = df_base.loc[df_base[coluna_percentil_temporal]>pct_split]
  return df_treino,df_teste

def PrepararBaseTreino(df_treino,list_features,target):
   df_X_treino = df_treino.loc[:,list_features]
   df_y_treino = df_treino.loc[:,[target]]
   return df_X_treino,df_y_treino

def TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo):
   alg = Algoritmo()
   alg.fit(df_X_treino,df_y_treino)
   return alg

def RealizarPredicaoTeste(df_test,list_features,target,modelo,modelo_number='1_1'):
   df_X_test = df_test[list_features]
   df_y_pred = modelo.predict(df_X_test)
   df_validacao = df_test.copy()
   df_validacao[f'{target}_modelo_{modelo_number}'] = df_y_pred
   return df_validacao

def CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva):
   metricas_modelo = {
      'MAE':MAE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'RMSE':RMSE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'R2':R2_determinacao(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
      'PSC_A':PSC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],psc_a_max_chuva),
      'PCC_A':PCC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pcc_a_erro),
      'PMC_A':PMC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pmc_a_erro,pmc_a_min_chuva)
   }

   return metricas_modelo

def SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_1,metricas_modelo_1_1,target,modelo_number,final_db=final_db):

    df_validacao_key_target_pred = df_validacao[list_key+[target,f'{target}_modelo_{modelo_number}']]

    final_db.execute(
    f"""
    CREATE OR REPLACE TABLE tb_validacao_modelo_{modelo_number} AS (
    SELECT * FROM df_validacao_key_target_pred)
    """)

    with open(f'modelos_finais/modelo_{modelo_number}.pkl','wb') as f:
        pickle.dump(modelo_1_1,f)

    with open(f'modelos_finais/metricas_{modelo_number}.json', 'w') as f:
        json.dump(metricas_modelo_1_1, f, indent=4)

    pass


## Modelo 1.1

In [ ]:
modelo_number = '1_1'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao'
list_features_modelo_1_1 = list_features_geoespaciais + list_features_estacoes
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo1()
df_treino,df_teste = SplitTreinoTeste_Modelo1(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_1_1,target)

# Treinando o modelo
modelo_1_1 = TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste(df_teste,list_features_modelo_1_1,target,modelo_1_1,modelo_number)

# Calculando Métricas
metricas_modelo_1_1 = CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_1,metricas_modelo_1_1,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 1.2

In [ ]:
modelo_number = '1_2'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao'
list_features_modelo_1_2 = list_features_geoespaciais + list_features_estacoes + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo1()
df_treino,df_teste = SplitTreinoTeste_Modelo1(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_1_2,target)

# Treinando o modelo
modelo_1_2 = TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste(df_teste,list_features_modelo_1_2,target,modelo_1_2,modelo_number)

# Calculando Métricas
metricas_modelo_1_2 = CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_2,metricas_modelo_1_2,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 1.3

In [4]:
modelo_number = '1_3'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao'
list_features_modelo_1_3 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo1()
df_treino,df_teste = SplitTreinoTeste_Modelo1(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_1_3,target)

# Treinando o modelo
modelo_1_3 = TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste(df_teste,list_features_modelo_1_3,target,modelo_1_3,modelo_number)

# Calculando Métricas
metricas_modelo_1_3 = CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_3,metricas_modelo_1_3,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 1.4

In [5]:
modelo_number = '1_4'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao'
list_features_modelo_1_4 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo1()
df_treino,df_teste = SplitTreinoTeste_Modelo1(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_1_4,target)

# Treinando o modelo
modelo_1_4 = TreinarAlgoritmo_Modelo1(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste(df_teste,list_features_modelo_1_4,target,modelo_1_4,modelo_number)

# Calculando Métricas
metricas_modelo_1_4 = CalcularMetricasTeste(df_validacao,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo1(df_validacao,modelo_1_4,metricas_modelo_1_4,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Comparações 

In [ ]:
list_modelo_numbers = ['1_1','1_2','1_3','1_4']

metricas_modelo_1 = {}
for modelo_number in list_modelo_numbers:
    with open(f'modelos_finais/metricas_{modelo_number}.json','r') as f:
        metricas_modelo_1[modelo_number] = json.load(f)

In [14]:
pd.DataFrame(metricas_modelo_1)

,1_1,1_2,1_3,1_4
MAE,5.460297,3.226563,2.856736,2.749362
RMSE,10.150194,7.859220,7.221627,7.092083
R2,0.030744,0.418903,0.509364,0.526808
PSC_A,6.428698,75.401540,77.500109,80.507026
PCC_A,11.641076,20.045932,25.463811,25.426790
PMC_A,0.473503,12.336413,18.179684,18.507399


# MODELOS 2

In [7]:
print("""Os modelos 2 são iguais aos modelos 1, porém com a variável target logarítmica. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação.\n
> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Target Logarítmica
> Testando diferentes combinações de variáveis explicativas
""")

Os modelos 2 são iguais aos modelos 1, porém com a variável target logarítmica. Esses modelos são compostos por um único algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação.

> Sem pré modelo de classificação
> Sem modelo especializado
> Apenas algoritmo de XGBoost
> Partição padrão
> Target Logarítmica
> Testando diferentes combinações de variáveis explicativas



In [8]:
def ImportBase_Modelo2(final_db=final_db,table_name='abt_base'):
    df = final_db.execute(f"""
    SELECT
        *
    FROM {table_name}
    """).fetch_df()

    return df

def SplitTreinoTeste_Modelo2(df_base,pct_split,coluna_percentil_temporal='percentil_temporal'):
  df_treino = df_base.loc[df_base[coluna_percentil_temporal]<=pct_split]
  df_teste = df_base.loc[df_base[coluna_percentil_temporal]>pct_split]
  return df_treino,df_teste

def PrepararBaseTreino(df_treino,list_features,target):
   df_X_treino = df_treino.loc[:,list_features]
   df_y_treino = df_treino.loc[:,[target]]
   return df_X_treino,df_y_treino

def TreinarAlgoritmo_Modelo2(df_X_treino,df_y_treino,Algoritmo):
   alg = Algoritmo()
   alg.fit(df_X_treino,df_y_treino)
   return alg

def RealizarPredicaoTeste_Modelo2(df_test,list_features,target,target_original,modelo,modelo_number):
   df_X_test = df_test[list_features]
   df_y_pred = modelo.predict(df_X_test)
   df_validacao = df_test.copy()
   df_validacao[f'{target}_modelo_{modelo_number}'] = df_y_pred
   df_validacao[f'{target_original}_modelo_{modelo_number}'] = np.exp(df_validacao[f'{target}_modelo_{modelo_number}'])
   return df_validacao

def CalcularMetricasTeste_Modelo2(df_validacao,target_original,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva):
   metricas_modelo = {
      'MAE':MAE(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}']),
      'RMSE':RMSE(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}']),
      'R2':R2_determinacao(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}']),
      'PSC_A':PSC_A(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}'],psc_a_max_chuva),
      'PCC_A':PCC_A(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}'],pcc_a_erro),
      'PMC_A':PMC_A(df_validacao[target_original],df_validacao[f'{target_original}_modelo_{modelo_number}'],pmc_a_erro,pmc_a_min_chuva)
   }

   return metricas_modelo

def SalvarValidacaoModeloMetricas_Modelo2(df_validacao,modelo,metricas_modelo,target,target_original,modelo_number,final_db=final_db):

    df_validacao_key_target_pred = df_validacao[list_key+[target_original,f'{target_original}_modelo_{modelo_number}']+[target,f'{target}_modelo_{modelo_number}']]

    final_db.execute(
    f"""
    CREATE OR REPLACE TABLE tb_validacao_modelo_{modelo_number} AS (
    SELECT * FROM df_validacao_key_target_pred)
    """)

    with open(f'modelos_finais/modelo_{modelo_number}.pkl','wb') as f:
        pickle.dump(modelo,f)

    with open(f'modelos_finais/metricas_{modelo_number}.json', 'w') as f:
        json.dump(metricas_modelo, f, indent=4)

    pass


## Modelo 2.1

In [9]:
modelo_number = '2_1'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao_log'
target_original = 'vl_precipitacao'
list_features_modelo_2_1 = list_features_geoespaciais + list_features_estacoes
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo2()
df_treino,df_teste = SplitTreinoTeste_Modelo2(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_2_1,target)

# Treinando o modelo
modelo_2_1 = TreinarAlgoritmo_Modelo2(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste_Modelo2(df_teste,list_features_modelo_2_1,target,target_original,modelo_2_1,modelo_number)

# Calculando Métricas
metricas_modelo_2_1 = CalcularMetricasTeste_Modelo2(df_validacao,target_original,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo2(df_validacao,modelo_2_1,metricas_modelo_2_1,target,target_original,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 2.2

In [11]:
modelo_number = '2_2'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao_log'
target_original = 'vl_precipitacao'
list_features_modelo_2_2 = list_features_geoespaciais + list_features_estacoes + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo2()
df_treino,df_teste = SplitTreinoTeste_Modelo2(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_2_2,target)

# Treinando o modelo
modelo_2_2 = TreinarAlgoritmo_Modelo2(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste_Modelo2(df_teste,list_features_modelo_2_2,target,target_original,modelo_2_2,modelo_number)

# Calculando Métricas
metricas_modelo_2_2 = CalcularMetricasTeste_Modelo2(df_validacao,target_original,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo2(df_validacao,modelo_2_2,metricas_modelo_2_2,target,target_original,modelo_number,final_db=final_db)

## Modelo 2.3

In [12]:
modelo_number = '2_3'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao_log'
target_original = 'vl_precipitacao'
list_features_modelo_2_3 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo2()
df_treino,df_teste = SplitTreinoTeste_Modelo2(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_2_3,target)

# Treinando o modelo
modelo_2_3 = TreinarAlgoritmo_Modelo2(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste_Modelo2(df_teste,list_features_modelo_2_3,target,target_original,modelo_2_3,modelo_number)

# Calculando Métricas
metricas_modelo_2_3 = CalcularMetricasTeste_Modelo2(df_validacao,target_original,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo2(df_validacao,modelo_2_3,metricas_modelo_2_3,target,target_original,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 2.4

In [13]:
modelo_number = '2_4'

# Parâmetros de treinamento
pct_train_test_split = 0.7
target = 'vl_precipitacao_log'
target_original = 'vl_precipitacao'
list_features_modelo_2_4 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo2()
df_treino,df_teste = SplitTreinoTeste_Modelo2(df_base,pct_train_test_split)
df_X_treino,df_y_treino = PrepararBaseTreino(df_treino,list_features_modelo_2_4,target)

# Treinando o modelo
modelo_2_4 = TreinarAlgoritmo_Modelo2(df_X_treino,df_y_treino,Algoritmo)

# Validando
df_validacao = RealizarPredicaoTeste_Modelo2(df_teste,list_features_modelo_2_4,target,target_original,modelo_2_4,modelo_number)

# Calculando Métricas
metricas_modelo_2_4 = CalcularMetricasTeste_Modelo2(df_validacao,target_original,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo2(df_validacao,modelo_2_4,metricas_modelo_2_4,target,target_original,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# MODELOS 3

In [2]:
print("""Os modelos 3 são iguais aos modelos 1_4, porém com separação entre modelo especializado e modelo geral. Esses modelos são compostos por dois algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação.\n
> Sem pré modelo de classificação
> Modelo especializado, testando diferentes thresholds
> Apenas algoritmo de XGBoost
> Partição padrão
> Target padrão
> Usando todas as variáveis explicativas
""")

Os modelos 3 são iguais aos modelos 1_4, porém com separação entre modelo especializado e modelo geral. Esses modelos são compostos por dois algoritmo de regressão, sempre usando XGBoost. A partição já é a padrão, de 70-30 por estação.

> Sem pré modelo de classificação
> Modelo especializado, testando diferentes thresholds
> Apenas algoritmo de XGBoost
> Partição padrão
> Target padrão
> Usando todas as variáveis explicativas



In [40]:
def ImportBase_Modelo3(final_db=final_db,table_name='abt_base'):
    df = final_db.execute(f"""
    SELECT
        *
    FROM {table_name}
    """).fetch_df()

    return df

def SepararBaseEspecializadoGeral_Modelo3(df_base,coluna_vl_prioridade_vizinha,threshold_modelo_especializado):
    df_base_especializado = df_base.loc[df_base[coluna_vl_prioridade_vizinha]>=threshold_modelo_especializado]
    df_base_geral = df_base.copy()
    return df_base_geral,df_base_especializado

def SplitTreinoTeste_Modelo3(df_base_geral,df_base_especializado,pct_split,coluna_percentil_temporal='percentil_temporal'):
  df_treino_especializado = df_base_especializado.loc[df_base_especializado[coluna_percentil_temporal]<=pct_split]
  df_teste_especializado = df_base_especializado.loc[df_base_especializado[coluna_percentil_temporal]>pct_split]

  df_treino_geral = df_base_geral.loc[df_base_geral[coluna_percentil_temporal]<=pct_split]
  df_teste_geral = df_base_geral.loc[df_base_geral[coluna_percentil_temporal]>pct_split]
  return df_treino_especializado,df_teste_especializado,df_treino_geral,df_teste_geral

def PrepararBaseTreino(df_treino,list_features,target):
   df_X_treino = df_treino.loc[:,list_features]
   df_y_treino = df_treino.loc[:,[target]]
   return df_X_treino,df_y_treino

def TreinarAlgoritmo_Modelo3(df_X_treino,df_y_treino,Algoritmo):
   alg = Algoritmo()
   alg.fit(df_X_treino,df_y_treino)
   return alg

def RealizarPredicaoTeste_Modelo3(df_test,list_features,target,modelo,modelo_number,tipo_modelo):
   df_X_test = df_test[list_features]
   df_y_pred = modelo.predict(df_X_test)
   df_validacao = df_test.copy()
   df_validacao[f'{target}_modelo_{modelo_number}_{tipo_modelo}'] = df_y_pred
   return df_validacao

def CalcularMetricasTeste_Modelo3(df_validacao_geral,df_validacao_especializado,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva):
    df_validacao_all = df_validacao_geral[['id_estacao','dt_medicao',f'vl_precipitacao_modelo_{modelo_number}_geral','vl_precipitacao']] \
        .merge(df_validacao_especializado[['id_estacao','dt_medicao',f'vl_precipitacao_modelo_{modelo_number}_especializado']],on=['id_estacao','dt_medicao'],how='left')

    df_validacao_all[f'vl_precipitacao_modelo_{modelo_number}'] = df_validacao_all[f'vl_precipitacao_modelo_{modelo_number}_especializado'].fillna(df_validacao_all[f'vl_precipitacao_modelo_{modelo_number}_geral'])
    df_validacao = df_validacao_all
    metricas_modelo = {
        'MAE':MAE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
        'RMSE':RMSE(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
        'R2':R2_determinacao(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}']),
        'PSC_A':PSC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],psc_a_max_chuva),
        'PCC_A':PCC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pcc_a_erro),
        'PMC_A':PMC_A(df_validacao[target],df_validacao[f'{target}_modelo_{modelo_number}'],pmc_a_erro,pmc_a_min_chuva)
    }
    return metricas_modelo,df_validacao

def SalvarValidacaoModeloMetricas_Modelo3(df_validacao,modelo_geral,modelo_especializado,metricas_modelo,target,modelo_number,final_db=final_db):

    df_validacao_key_target_pred = df_validacao[list_key+[target,f'{target}_modelo_{modelo_number}']]

    final_db.execute(
    f"""
    CREATE OR REPLACE TABLE tb_validacao_modelo_{modelo_number} AS (
    SELECT * FROM df_validacao_key_target_pred)
    """)

    with open(f'modelos_finais/modelo_{modelo_number}_geral.pkl','wb') as f:
        pickle.dump(modelo_geral,f)

    with open(f'modelos_finais/modelo_{modelo_number}_especializado.pkl','wb') as f:
        pickle.dump(modelo_especializado,f)

    with open(f'modelos_finais/metricas_{modelo_number}.json', 'w') as f:
        json.dump(metricas_modelo, f, indent=4)

    pass


## Modelo 3.1

In [ ]:
modelo_number = '3_1'

# Parâmetros de treinamento
pct_train_test_split = 0.7
threshold_modelo_especializado = 0.5
target = 'vl_precipitacao'
coluna_vl_prioridade_vizinha = 'vl_prioridade_vizinha_1'
list_features_modelo_3 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo3()

df_base_geral,df_base_especializado = SepararBaseEspecializadoGeral_Modelo3(df_base,coluna_vl_prioridade_vizinha,threshold_modelo_especializado)
df_treino_especializado,df_teste_especializado,df_treino_geral,df_teste_geral = SplitTreinoTeste_Modelo3(df_base_geral,df_base_especializado,pct_train_test_split,coluna_percentil_temporal='percentil_temporal')

df_X_treino_especializado,df_y_treino_especializado = PrepararBaseTreino(df_treino_especializado,list_features_modelo_3,target)
df_X_treino_geral,df_y_treino_geral = PrepararBaseTreino(df_teste_especializado,list_features_modelo_3,target)

# Treinando os modelos especializados e geral
modelo_3_1_especializado = TreinarAlgoritmo_Modelo3(df_X_treino_especializado,df_y_treino_especializado,Algoritmo)
modelo_3_1_geral = TreinarAlgoritmo_Modelo3(df_X_treino_geral,df_y_treino_geral,Algoritmo)

# Validando
df_validacao_especializado = RealizarPredicaoTeste_Modelo3(df_teste_especializado,list_features_modelo_3,target,modelo_3_1_especializado,modelo_number,'especializado')
df_validacao_geral = RealizarPredicaoTeste_Modelo3(df_teste_geral,list_features_modelo_3,target,modelo_3_1_geral,modelo_number,'geral')

# Calculando Métricas
metricas_modelo_3_1,df_validacao_all = CalcularMetricasTeste_Modelo3(df_validacao_geral,df_validacao_especializado,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo3(df_validacao_all,modelo_3_1_geral,modelo_3_1_especializado,metricas_modelo_3_1,target,modelo_number,final_db=final_db)

## Modelo 3.2

In [ ]:
modelo_number = '3_2'

# Parâmetros de treinamento
pct_train_test_split = 0.7
threshold_modelo_especializado = 0.75
target = 'vl_precipitacao'
coluna_vl_prioridade_vizinha = 'vl_prioridade_vizinha_1'
list_features_modelo_3 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo3()

df_base_geral,df_base_especializado = SepararBaseEspecializadoGeral_Modelo3(df_base,coluna_vl_prioridade_vizinha,threshold_modelo_especializado)
df_treino_especializado,df_teste_especializado,df_treino_geral,df_teste_geral = SplitTreinoTeste_Modelo3(df_base_geral,df_base_especializado,pct_train_test_split,coluna_percentil_temporal='percentil_temporal')

df_X_treino_especializado,df_y_treino_especializado = PrepararBaseTreino(df_treino_especializado,list_features_modelo_3,target)
df_X_treino_geral,df_y_treino_geral = PrepararBaseTreino(df_teste_especializado,list_features_modelo_3,target)

# Treinando os modelos especializados e geral
modelo_3_2_especializado = TreinarAlgoritmo_Modelo3(df_X_treino_especializado,df_y_treino_especializado,Algoritmo)
modelo_3_2_geral = TreinarAlgoritmo_Modelo3(df_X_treino_geral,df_y_treino_geral,Algoritmo)

# Validando
df_validacao_especializado = RealizarPredicaoTeste_Modelo3(df_teste_especializado,list_features_modelo_3,target,modelo_3_2_especializado,modelo_number,'especializado')
df_validacao_geral = RealizarPredicaoTeste_Modelo3(df_teste_geral,list_features_modelo_3,target,modelo_3_2_geral,modelo_number,'geral')

# Calculando Métricas
metricas_modelo_3_2,df_validacao_all = CalcularMetricasTeste_Modelo3(df_validacao_geral,df_validacao_especializado,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo3(df_validacao_all,modelo_3_2_geral,modelo_3_2_especializado,metricas_modelo_3_2,target,modelo_number,final_db=final_db)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Modelo 3.3

In [ ]:
modelo_number = '3_3'

# Parâmetros de treinamento
pct_train_test_split = 0.7
threshold_modelo_especializado = 0.9
target = 'vl_precipitacao'
coluna_vl_prioridade_vizinha = 'vl_prioridade_vizinha_1'
list_features_modelo_3 = list_features_geoespaciais + list_features_estacoes + list_features_vizinhas + list_features_produtos
Algoritmo = XGBRegressor

# Parâmetros de métricas
psc_a_max_chuva = 1
pcc_a_erro = 1
pmc_a_erro = 5
pmc_a_min_chuva = 20

# Preparando base
df_base = ImportBase_Modelo3()

df_base_geral,df_base_especializado = SepararBaseEspecializadoGeral_Modelo3(df_base,coluna_vl_prioridade_vizinha,threshold_modelo_especializado)
df_treino_especializado,df_teste_especializado,df_treino_geral,df_teste_geral = SplitTreinoTeste_Modelo3(df_base_geral,df_base_especializado,pct_train_test_split,coluna_percentil_temporal='percentil_temporal')

df_X_treino_especializado,df_y_treino_especializado = PrepararBaseTreino(df_treino_especializado,list_features_modelo_3,target)
df_X_treino_geral,df_y_treino_geral = PrepararBaseTreino(df_teste_especializado,list_features_modelo_3,target)

# Treinando os modelos especializados e geral
modelo_3_3_especializado = TreinarAlgoritmo_Modelo3(df_X_treino_especializado,df_y_treino_especializado,Algoritmo)
modelo_3_3_geral = TreinarAlgoritmo_Modelo3(df_X_treino_geral,df_y_treino_geral,Algoritmo)

# Validando
df_validacao_especializado = RealizarPredicaoTeste_Modelo3(df_teste_especializado,list_features_modelo_3,target,modelo_3_3_especializado,modelo_number,'especializado')
df_validacao_geral = RealizarPredicaoTeste_Modelo3(df_teste_geral,list_features_modelo_3,target,modelo_3_3_geral,modelo_number,'geral')

# Calculando Métricas
metricas_modelo_3_3,df_validacao_all = CalcularMetricasTeste_Modelo3(df_validacao_geral,df_validacao_especializado,target,modelo_number,psc_a_max_chuva,pcc_a_erro,pmc_a_erro,pmc_a_min_chuva)

# Salvando modelo, base de validação e métricas
SalvarValidacaoModeloMetricas_Modelo3(df_validacao_all,modelo_3_3_geral,modelo_3_3_especializado,metricas_modelo_3_3,target,modelo_number,final_db=final_db)

## Comparações

In [ ]:
list_modelo_numbers = ['1_4','2_4','3_1','3_2','3_3']

metricas_modelo_1_2_3 = {}
for modelo_number in list_modelo_numbers:
    with open(f'modelos_finais/metricas_{modelo_number}.json','r') as f:
        metricas_modelo_1_2_3[modelo_number] = json.load(f)

In [ ]:
pd.DataFrame(metricas_modelo_1_2_3).T

,MAE,RMSE,R2,PSC_A,PCC_A,PMC_A
1_4,2.749362,7.092083,0.526808,80.507026,25.426790,18.507399
2_4,3.170629,7.535428,0.465798,6.937751,24.771411,9.711914
3_1,2.731023,7.141533,0.520186,82.139563,26.123419,19.915245
